 ## Data << Extreme_Value_Theory.zip

In [1]:
cd C:\Users\USER\수정용_금융리스크관리\Extreme_Value_Theory\EVT

C:\Users\USER\수정용_금융리스크관리\Extreme_Value_Theory\EVT


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
from scipy.optimize import root

In [3]:
def preprocessing_lossdata(path_price,columns_name = 'Close') :
    data = pd.read_csv('F567C.s2020.HW5.data.csv', index_col=0)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data.index = pd.to_datetime(data.index)
    data = data.astype(np.float64)
    my_loss = -data[columns_name].pct_change().iloc[1:]
    return my_loss
path_price = 'F567C.s2020.HW5.data.csv'
my_loss = preprocessing_lossdata(path_price)

 ### MLE of GPD parameters

#### Generalized Pareto Distribution
$$
\begin{array}{c}
G(\xi, \beta, X, u) = 1 - ( 1+ \xi \frac{X-u}{\beta})^{-1/\xi} \\
g(\xi, \beta, X, u) = \frac{1}{\beta} ( 1 + \xi \frac{X-u}{\beta})^{- \frac{1}{\xi} -1}
\end{array}
$$
#### Maximum Likelihood Estimator
$$
\begin{array}{c}
 maximize \  \sum_{i=1}^{n} (ln \frac{1}{\beta} ( 1 + \xi \frac{X_i-u}{\beta})^{- \frac{1}{\xi} -1} )
\end{array}
$$

 #### Probability function and VaR, ES
$$
\begin{array}{c}
\alpha = P(X>VaR) = P(X>u+x) \\ \\ = P(X>u)(1+\xi \frac{x}{\beta})^{-1/\xi} = P(X>u)(1+\xi \frac{VaR-u}{\beta})^{-1/\xi}
\\
\\
VaR = u + \frac{\beta}{\xi} ( (\frac{\alpha}{P(X>u)})^{-\xi}-1) \\ \\
ES  = \frac{VaR + \beta - \xi u}{1-\xi} 
\end{array}
$$

In [4]:
class GPD :
    def cdf(X,beta_xi_array, u) :
        ##################################
        ## beta_xi_array = [beta, xi] ##
        ##################################
        return 1-(1+beta_xi_array[1]*(X-u)/beta_xi_array[0])**(-1/beta_xi_array[1])
    def pdf(X,beta_xi_array, u) :
        ##################################
        ## beta_xi_array = [beta, xi] ##
        ##################################
        return 1/beta_xi_array[0] * (1+ beta_xi_array[1]*(X-u)/beta_xi_array[0])**(-1/beta_xi_array[1]-1)
    def log_pdf_sum(X,beta_xi_array,u) :
        ###########################
        ## Sum of log likelihood ##
        ###########################
        extreme_X = X[X>u]
        likelihood = GPD.pdf(extreme_X,beta_xi_array,u)
        return np.log(likelihood).sum()
    
    def estimate_pareto_params(X, u) :
        ###########################
        ## Estimate beta and xi  ##
        ###########################
        beta0 = 0.006
        xi0 = 0.15
        beta_xi = np.array([beta0 , xi0])
        extreme_X = X[X>u]
        minus_log_likelihood = lambda beta_xi : -1 * GPD.log_pdf_sum(X,beta_xi,u)
        bnds =  ((0.00001,10), (0.00001,10))
        x0 = (beta_xi[0], beta_xi[1])
        res = minimize(minus_log_likelihood, x0,method = 'SLSQP', bounds = bnds)
        return res

def Calculate_VaR_and_ES_using_EVT(loss_data, threshold, alpha) :
    #########################################
    ## loss_data is negative return data    #
    ## alpha is a significance level        #
    #########################################
    u = threshold
    beta_xi_array = GPD.estimate_pareto_params(my_loss, u).x
    P_X_over_u = len(loss_data[loss_data>u]) / len(loss_data)
    VaR = u + beta_xi_array[0]/beta_xi_array[1] * ( (alpha/P_X_over_u )**(-beta_xi_array[1]) -1  )
    ES = (VaR + beta_xi_array[0] - beta_xi_array[1] * u)/(1-beta_xi_array[1])
    return VaR, ES

In [5]:
Calculate_VaR_and_ES_using_EVT(my_loss, 0.01, 0.001)

(0.06174415093768072, 0.07957654256739141)